In [1]:
# Run some setup code for this notebook.
import pandas as pd

# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2


In [2]:
# 数据整理
import numpy as np 
import pandas as pd 

# 可视化
import matplotlib.pyplot as plt    
import seaborn as sns
import warnings

#预测
from sklearn import metrics
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix 
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
#from sklearn.preprocessing import MinMaxScaler

from sklearn.naive_bayes import CategoricalNB
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier 
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBRegressor as XGBR
from sklearn.svm import SVC

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

from xgboost import XGBRegressor as XGBR

#格式
%matplotlib inline
plt.rcParams['font.sans-serif']=['SimHei']#显示中文
plt.rcParams['axes.unicode_minus']=False  #正常显示负号

In [94]:
def load_data(path):
    df = pd.read_json(path)
    df_expanded = df['user'].apply(lambda x: pd.Series(x))
    df = pd.concat([df.drop('user', axis=1), df_expanded], axis=1)
    return df

data_train = load_data("./data/train.json") 
data_test = load_data("./data/test.json")

In [95]:
data_train

,created_at,label,id,id_str,name,screen_name,location,description,url,entities,...,profile_sidebar_fill_color,profile_text_color,profile_use_background_image,has_extended_profile,default_profile,default_profile_image,following,follow_request_sent,notifications,translator_type
0,2019-01-20 03:36:01+00:00,bot,1318284638,1318284638,Syed Haider Naqvi,HNakvi,Islamabad,,None,{'description': {'urls': []}},...,DDEEF6,333333,True,False,True,False,False,False,False,none
1,2019-01-21 21:36:00+00:00,bot,17442457,17442457,Cleveland Clinic MD,CleClinicMD,"Cleveland, Ohio, United States",News and insights for physicians and clinical ...,https://t.co/IxMra2OEey,{'url': {'urls': [{'url': 'https://t.co/IxMra2...,...,D6E9F6,333333,True,False,False,False,False,False,False,none
2,2016-10-11 13:40:17+00:00,bot,102069605,102069605,2NE1 TV,2NE1tv,Seoul,We tweet about 2NE1's video and photo. It’s 2N...,None,{'description': {'urls': []}},...,E3E2DE,634047,True,False,False,False,False,False,False,none
3,2019-01-21 18:42:38+00:00,bot,1367759732,1367759732,Alastair de kretser,Alastairkretser,England,"Travel, tech and stuff 😀",None,{'description': {'urls': []}},...,DDEEF6,333333,True,True,False,False,False,False,False,none
4,2018-11-13 04:09:04+00:00,bot,2927879268,2927879268,Gran Horóscopo,GranHoroscopo,México - Argentina,Tu mejor prediccion diaria ORIGINAL para todos...,https://t.co/rDVUTyCn9E,{'url': {'urls': [{'url': 'https://t.co/rDVUTy...,...,DDEEF6,333333,True,False,True,False,False,False,False,regular
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1981,2017-07-16 15:45:38+00:00,human,464353156,464353156,Biα ♐️,Biia_Amaraal1,Artur Nogueira,👻 bia.amarall/ ♐️,None,{'description': {'urls': []}},...,DDEEF6,333333,True,True,False,False,False,False,False,none
1982,2019-01-21 21:55:19+00:00,human,214891135,214891135,Brooke Marie,BrookeMarieCam,🤱🏼,Welcome! I specialize in Adult (18+) Custom Vi...,https://t.co/uAZ5LhUnNT,{'url': {'urls': [{'url': 'https://t.co/uAZ5Lh...,...,FFFFFF,333333,False,True,False,False,False,False,False,none
1983,2019-01-21 02:41:37+00:00,human,247977754,247977754,Bianca 🌴,biancabizb_,"Arauco, Chile",18 • 🐾💘,https://t.co/Qfnt0mOUG3,{'url': {'urls': [{'url': 'https://t.co/Qfnt0m...,...,DDEEF6,333333,True,True,False,False,False,False,False,regular
1984,2019-01-21 12:52:24+00:00,human,274251944,274251944,Cameen,Cameen23,"New England, Brooklyn-NY",New England Music Awards Top five Artist Of Th...,https://t.co/vhcGaX7t9m,{'url': {'urls': [{'url': 'https://t.co/vhcGaX...,...,EFEFEF,333333,True,False,False,False,False,False,False,none


In [96]:
#数据基本信息及查重
data_train.info()
len(data_train.index.unique())
#与实际数据数相同，说明每一行数据的index唯一且不相重

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1986 entries, 0 to 1985
Data columns (total 44 columns):
 #   Column                              Non-Null Count  Dtype              
---  ------                              --------------  -----              
 0   created_at                          1986 non-null   datetime64[ns, UTC]
 1   label                               1986 non-null   object             
 2   id                                  1986 non-null   int64              
 3   id_str                              1986 non-null   object             
 4   name                                1986 non-null   object             
 5   screen_name                         1986 non-null   object             
 6   location                            1986 non-null   object             
 7   description                         1986 non-null   object             
 8   url                                 1223 non-null   object             
 9   entities                            1986 

1986

In [97]:
#删除全空的属性列
data_train.drop(['utc_offset','time_zone'], axis=1, inplace=True) 

In [98]:
scaler = StandardScaler()
# 对int类型属性进行标准化
data_train[['statuses_count', 'followers_count', 'friends_count','favourites_count','listed_count']] = scaler.fit_transform(data_train[['statuses_count', 'followers_count', 'friends_count','favourites_count','listed_count']])


In [99]:
#多了一个created_at，删除后面那个（前面那个是时间戳）
data_train=data_train.loc[:,~data_train.columns.duplicated()]
data_train

,created_at,label,id,id_str,name,screen_name,location,description,url,entities,...,profile_sidebar_fill_color,profile_text_color,profile_use_background_image,has_extended_profile,default_profile,default_profile_image,following,follow_request_sent,notifications,translator_type
0,2019-01-20 03:36:01+00:00,bot,1318284638,1318284638,Syed Haider Naqvi,HNakvi,Islamabad,,None,{'description': {'urls': []}},...,DDEEF6,333333,True,False,True,False,False,False,False,none
1,2019-01-21 21:36:00+00:00,bot,17442457,17442457,Cleveland Clinic MD,CleClinicMD,"Cleveland, Ohio, United States",News and insights for physicians and clinical ...,https://t.co/IxMra2OEey,{'url': {'urls': [{'url': 'https://t.co/IxMra2...,...,D6E9F6,333333,True,False,False,False,False,False,False,none
2,2016-10-11 13:40:17+00:00,bot,102069605,102069605,2NE1 TV,2NE1tv,Seoul,We tweet about 2NE1's video and photo. It’s 2N...,None,{'description': {'urls': []}},...,E3E2DE,634047,True,False,False,False,False,False,False,none
3,2019-01-21 18:42:38+00:00,bot,1367759732,1367759732,Alastair de kretser,Alastairkretser,England,"Travel, tech and stuff 😀",None,{'description': {'urls': []}},...,DDEEF6,333333,True,True,False,False,False,False,False,none
4,2018-11-13 04:09:04+00:00,bot,2927879268,2927879268,Gran Horóscopo,GranHoroscopo,México - Argentina,Tu mejor prediccion diaria ORIGINAL para todos...,https://t.co/rDVUTyCn9E,{'url': {'urls': [{'url': 'https://t.co/rDVUTy...,...,DDEEF6,333333,True,False,True,False,False,False,False,regular
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1981,2017-07-16 15:45:38+00:00,human,464353156,464353156,Biα ♐️,Biia_Amaraal1,Artur Nogueira,👻 bia.amarall/ ♐️,None,{'description': {'urls': []}},...,DDEEF6,333333,True,True,False,False,False,False,False,none
1982,2019-01-21 21:55:19+00:00,human,214891135,214891135,Brooke Marie,BrookeMarieCam,🤱🏼,Welcome! I specialize in Adult (18+) Custom Vi...,https://t.co/uAZ5LhUnNT,{'url': {'urls': [{'url': 'https://t.co/uAZ5Lh...,...,FFFFFF,333333,False,True,False,False,False,False,False,none
1983,2019-01-21 02:41:37+00:00,human,247977754,247977754,Bianca 🌴,biancabizb_,"Arauco, Chile",18 • 🐾💘,https://t.co/Qfnt0mOUG3,{'url': {'urls': [{'url': 'https://t.co/Qfnt0m...,...,DDEEF6,333333,True,True,False,False,False,False,False,regular
1984,2019-01-21 12:52:24+00:00,human,274251944,274251944,Cameen,Cameen23,"New England, Brooklyn-NY",New England Music Awards Top five Artist Of Th...,https://t.co/vhcGaX7t9m,{'url': {'urls': [{'url': 'https://t.co/vhcGaX...,...,EFEFEF,333333,True,False,False,False,False,False,False,none


In [100]:
#发现所有数据项的'follow_request_sent'值都为True，故删除,其它几个同理
duplicate_rows = data_train[data_train['follow_request_sent']==False]
duplicate_rows['label'].value_counts()

human    1130
bot       856
Name: label, dtype: int64

In [101]:
duplicate_rows = data_train[data_train['contributors_enabled']==False]
duplicate_rows['label'].value_counts()

human    1130
bot       856
Name: label, dtype: int64

In [102]:
duplicate_rows = data_train[data_train['following']==True]
duplicate_rows['label'].value_counts()
#'following','is_translator'取值为True的时候，只有两个数据，且label取值一致，猜测是不是可以利用

human    2
Name: label, dtype: int64

In [103]:
#url大部分是重复的，直接删除这个属性
#发现所有数据项的'follow_request_sent','notifications','protected','contributors_enabled'值都为True，故删除这些属性
data_train.drop(['url','follow_request_sent','notifications','protected','contributors_enabled'], axis=1, inplace=True) 

C:\Users\CindyWu\AppData\Local\Temp\ipykernel_2204\3558100619.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_train.drop(['url','follow_request_sent','notifications','protected','contributors_enabled'], axis=1, inplace=True)


In [104]:
#删除没有任何重复的数据项，对于预测没有帮助
data_train.drop(['id','id_str', 'name','screen_name'], axis=1, inplace=True) 

C:\Users\CindyWu\AppData\Local\Temp\ipykernel_2204\3522672398.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_train.drop(['id','id_str', 'name','screen_name'], axis=1, inplace=True)


In [105]:
#把'profile_background_image_url_https'中的内容单独提取出来
theme_mapping = {
    'https://abs.twimg.com/images/themes/theme1/bg.png': 'theme1',
    'https://abs.twimg.com/images/themes/theme2/bg.gif': 'theme2',
    'https://abs.twimg.com/images/themes/theme3/bg.gif': 'theme3',
    'https://abs.twimg.com/images/themes/theme4/bg.gif': 'theme4',
    'https://abs.twimg.com/images/themes/theme5/bg.gif': 'theme5',
    'https://abs.twimg.com/images/themes/theme6/bg.gif': 'theme6',
    'https://abs.twimg.com/images/themes/theme7/bg.gif': 'theme7',
    'https://abs.twimg.com/images/themes/theme8/bg.gif': 'theme8',
    'https://abs.twimg.com/images/themes/theme9/bg.gif': 'theme9',
    'https://abs.twimg.com/images/themes/theme10/bg.gif': 'theme10',
    'https://abs.twimg.com/images/themes/theme11/bg.gif': 'theme11',
    'https://abs.twimg.com/images/themes/theme12/bg.gif': 'theme12',
    'https://abs.twimg.com/images/themes/theme13/bg.gif': 'theme13',
    'https://abs.twimg.com/images/themes/theme14/bg.gif': 'theme14',
    'https://abs.twimg.com/images/themes/theme15/bg.png': 'theme15',
    'https://abs.twimg.com/images/themes/theme16/bg.gif': 'theme16',
    'https://abs.twimg.com/images/themes/theme17/bg.gif': 'theme17',
    'https://abs.twimg.com/images/themes/theme18/bg.gif': 'theme18',
    'https://abs.twimg.com/images/themes/theme19/bg.gif': 'theme19',
    'https://abs.twimg.com/images/themes/theme20/bg.png': 'theme20'
}

# Map the values in the 'theme' column based on the 'profile_background_image_url_https' column
data_train['background_theme'] = data_train['profile_background_image_url_https'].map(theme_mapping)

# Print the counts for each theme
print(data_train['background_theme'].value_counts(sort=False))

theme1     1169
theme3       16
theme7       26
theme9      107
theme14     214
theme4       48
theme10      54
theme8       14
theme12      17
theme5       18
theme6       22
theme18      34
theme15      53
theme11      34
theme13      23
theme16      33
theme2       16
theme17      22
theme19      17
theme20       1
Name: background_theme, dtype: int64


C:\Users\CindyWu\AppData\Local\Temp\ipykernel_2204\699265238.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_train['background_theme'] = data_train['profile_background_image_url_https'].map(theme_mapping)


In [106]:
#处理created_at时间戳，变成3个属性：年，月，日
data_train['created_at'] = pd.to_datetime(data_train['created_at'])
data_train['created_at_year'] = data_train['created_at'].dt.year
data_train['created_at_month'] = data_train['created_at'].dt.month
data_train['created_at_day'] = data_train['created_at'].dt.day
data_train.drop(['created_at'], axis=1, inplace=True) 

C:\Users\CindyWu\AppData\Local\Temp\ipykernel_2204\224732014.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_train['created_at'] = pd.to_datetime(data_train['created_at'])
C:\Users\CindyWu\AppData\Local\Temp\ipykernel_2204\224732014.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_train['created_at_year'] = data_train['created_at'].dt.year
C:\Users\CindyWu\AppData\Local\Temp\ipykernel_2204\224732014.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataF

In [107]:
#profile_banner_url属性有192个缺失，发现这些缺失了的数据项时bot的几率大于为human的几率，所以单独处理为一个属性'profile_banner_url_dup'
duplicate_rows = data_train[data_train['profile_banner_url'].duplicated(keep=False)]
duplicate_rows['label'].value_counts()

bot      119
human     73
Name: label, dtype: int64

In [108]:
duplicate_rows = data_train[data_train['profile_banner_url'].duplicated(keep=False)]
data_train.loc[duplicate_rows.index, 'profile_banner_url_dup'] = True
data_train.loc[~data_train.index.isin(duplicate_rows.index), 'profile_banner_url_dup'] = False

C:\Users\CindyWu\AppData\Local\Temp\ipykernel_2204\2276091969.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_train.loc[duplicate_rows.index, 'profile_banner_url_dup'] = True


In [109]:
duplicate_rows = data_train[data_train['favourites_count']>7]
duplicate_rows['label'].value_counts()

bot    6
Name: label, dtype: int64

In [22]:
#对lang属性独热编码并且预测，准确率约为0.66
data_lang=data_train[['lang','label']]
one_hot = pd.get_dummies(data_lang['lang'])
data_lang = pd.concat([data_lang, one_hot], axis=1)
data_lang = data_lang.drop('lang', axis=1)
X_1 = data_lang[data_lang.columns[~data_lang.columns.isin(["label"])]]
y_1 = data_lang.label
# dividing X, y into train and test data 
X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X_1,y_1, random_state = 1,train_size=0.9) 

In [22]:
def acc(y_test,y_pred):
    k=0
    for i in range(len(y_pred)):
        if y_pred[i]==list(y_test)[i]:
            k+=1
    ACC=k/len(y_pred)
    return(ACC)

In [24]:
def logistic(X_train, y_train, X_test, y_test):
    # 定义逻辑回归模型
    logreg = LogisticRegression(max_iter=100000)

    # 定义超参数的候选值
    param = {
        'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
        'penalty': ['l2']
    }

    # 使用GridSearchCV进行参数搜索
    grid_search = GridSearchCV(logreg, param, cv=3, scoring='accuracy')
    grid_search.fit(X_train, y_train)

    # 输出最佳参数
    best_params = grid_search.best_params_
    print("最佳参数：", best_params)

    # 在测试集上评估性能
    y_pred = grid_search.predict(X_test)

    # 输出 准确性
    f1 = f1_score(y_test, y_pred, average='macro')
    accuracy = accuracy_score(y_test, y_pred)
    print('逻辑回归算法:')
    print("f1_score标准：", f1)
    print("acc标准", accuracy)

In [20]:
def random_forest(X_train, y_train, X_test, y_test):
    # 定义随机森林分类器模型
    forest = RandomForestClassifier()

    # 定义超参数的候选值
    param = {
        'n_estimators': [50, 100, 200],
        'criterion':['gini','entropy'],
        'max_depth': [None, 10, 20],
        'min_samples_split': [2, 5, 10],
        'bootstrap':[True,False],
        'min_samples_leaf': [1, 2, 4]
    }

    # 使用GridSearchCV进行参数搜索
    grid_search = GridSearchCV(forest, param, cv=3, scoring='accuracy')
    grid_search.fit(X_train, y_train)

    # 输出最佳参数
    best_params = grid_search.best_params_
    print("随机森林算法：")
    print("最佳参数：", best_params)

    # 在测试集上评估性能
    y_pred = grid_search.predict(X_test)

    # 输出 F1 Score 和准确性
    f1 = f1_score(y_test, y_pred, average='macro')
    accuracy = accuracy_score(y_test, y_pred)

    print("f1_score标准：", f1)
    print("acc标准:", accuracy)

In [21]:
def knn(X_train, y_train, X_test, y_test):
    # 定义KNN模型
    knn = KNeighborsClassifier()

    # 定义超参数的候选值
    param = {
        'n_neighbors': [3, 5, 7, 9, 12],
        'weights': ['uniform', 'distance'],
        'p': [1, 2]
    }

    # 使用GridSearchCV进行参数搜索
    grid_search = GridSearchCV(knn, param, cv=3, scoring='accuracy')
    grid_search.fit(X_train, y_train)

    # 输出最佳参数
    best_params = grid_search.best_params_
    print("最佳参数：", best_params)

    # 在测试集上评估性能
    y_pred = grid_search.predict(X_test)

    # 输出 F1 Score 和准确性
    f1 = f1_score(y_test, y_pred, average='macro')
    accuracy = accuracy_score(y_test, y_pred)

    print("KNN算法：")
    print("f1_score标准：", f1)
    print("acc标准", accuracy)

In [50]:
def decision(X_train, y_train, X_test, y_test):
    # 定义决策树模型
    model = DecisionTreeClassifier(random_state=42)

    # 定义超参数的候选值
    param = {
        'criterion': ['gini', 'entropy'],
        'max_depth': [2, 3, 5, 6, 7, 8, 10],
        'min_samples_split': [1, 2, 5, 8, 10],
        'min_samples_leaf': [1, 2, 3, 4]
    }

    # 使用GridSearchCV进行参数搜索
    grid_search = GridSearchCV(model, param, cv=3, scoring='accuracy')
    grid_search.fit(X_train, y_train)

    # 输出最佳参数
    best_params = grid_search.best_params_
    print("最佳参数：", best_params)

    # 在测试集上评估性能
    y_pred = grid_search.predict(X_test)

    # 输出 F1 Score 和准确性
    f1 = f1_score(y_test, y_pred, average='weighted')
    accuracy = accuracy_score(y_test, y_pred)

    print("决策树算法：")
    print("f1_score标准：", f1)
    print("acc标准", accuracy)


In [51]:
def shenjing(X_train, y_train, X_test, y_test):
    # 定义神经网络模型
    classifier = MLPClassifier(max_iter=2000)

    # 定义超参数的候选值
    param = {
        'hidden_layer_sizes': [ (50,), (30, 20), (50, 30)],
        'activation': ['tanh', 'relu','logistic',],
        'alpha': [0.001, 0.01],
        'learning_rate': ['constant', 'invscaling', 'adaptive'],
    }

    # 使用GridSearchCV进行参数搜索
    grid_search = GridSearchCV(classifier, param, cv=3, scoring='accuracy')
    grid_search.fit(X_train, y_train)
    print("神经网络算法：")

    # 输出最佳参数
    best_params = grid_search.best_params_
    print("最佳参数：", best_params)

    # 在测试集上评估性能
    y_pred = grid_search.predict(X_test)
    test_accuracy = accuracy_score(y_test, y_pred)
    print("准确率：", test_accuracy)


In [52]:
def svm(X_train, y_train, X_test, y_test):
    svm_classifier = SVC()
    param = {
        'C': [0.01, 0.1, 1, 10, 100, 1000],
        'kernel': ['linear', 'rbf', 'poly', 'sigmoid']
    }

    grid_search = GridSearchCV(svm_classifier, param, cv=3, scoring='accuracy')
    grid_search.fit(X_train, y_train)

    best_params = grid_search.best_params_
    print("SVM算法:")
    print("最佳参数：", best_params)

    best_svm_classifier = SVC(**best_params)
    best_svm_classifier.fit(X_train, y_train)
    y_pred_svm = best_svm_classifier.predict(X_test)
    
    accuracy_svm = accuracy_score(y_test, y_pred_svm)
    print("最佳准确率：", accuracy_svm)

In [46]:
def test_acc(X_train_1,y_train_1,X_test_1,y_test_1):
    logistic(X_train_1,y_train_1,X_test_1,y_test_1)
    random_forest(X_train_1,y_train_1,X_test_1,y_test_1)
    #bayes(X_train_1,y_train_1,X_test_1,y_test_1)
    knn(X_train_1,y_train_1,X_test_1,y_test_1)
    decision(X_train_1,y_train_1,X_test_1,y_test_1)
    shenjing(X_train_1,y_train_1,X_test_1,y_test_1)
    svm(X_train_1,y_train_1,X_test_1,y_test_1)

In [111]:
data=data_train[['geo_enabled','is_translator','profile_banner_url_dup','following','lang','label','statuses_count', 'followers_count', 'friends_count','favourites_count','listed_count']]
one_hot = pd.get_dummies(data['lang'])
data= pd.concat([data, one_hot], axis=1)
data = data.drop('lang', axis=1)
X_00 = data[data.columns[~data.columns.isin(["label"])]]
y_00 = data['label']
X_train_00, X_test_00, y_train_00, y_test_00 = train_test_split(X_00,y_00, random_state = 1,train_size=0.9) 
test_acc(X_train_00, y_train_00, X_test_00, y_test_00)

最佳参数： {'criterion': 'entropy', 'max_depth': 6, 'min_samples_leaf': 3, 'min_samples_split': 8}
决策树算法：
f1_score标准： 0.7307268636097657
acc标准 0.7336683417085427


In [112]:
shenjing(X_train_00, y_train_00, X_test_00, y_test_00)

E:\anaconda\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:684: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (2000) reached and the optimization hasn't converged yet.
  warnings.warn(
E:\anaconda\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:684: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (2000) reached and the optimization hasn't converged yet.
  warnings.warn(
E:\anaconda\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:684: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (2000) reached and the optimization hasn't converged yet.
  warnings.warn(
E:\anaconda\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:684: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (2000) reached and the optimization hasn't converged yet.
  warnings.warn(
E:\anaconda\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: UserWarning: Training interrupted by

神经网络算法：
最佳参数： {'activation': 'tanh', 'alpha': 0.01, 'hidden_layer_sizes': (50,), 'learning_rate': 'invscaling'}
准确率： 0.7839195979899497


In [117]:
X_test_00.drop(['ca','nl','pl','ru'],axis=1,inplace=True)
svm(X_train_00, y_train_00, X_test_00, y_test_00)

SVM算法:
最佳参数： {'C': 0.1, 'kernel': 'linear'}
最佳准确率： 0.7386934673366834


In [113]:
#处理测试集，并预测测试集的结果
duplicate_rows = data_test[data_test['profile_banner_url'].duplicated(keep=False)]
data_test.loc[duplicate_rows.index, 'profile_banner_url_dup'] = True
data_test.loc[~data_test.index.isin(duplicate_rows.index), 'profile_banner_url_dup'] = False
data_test[['statuses_count', 'followers_count', 'friends_count','favourites_count','listed_count']] = scaler.fit_transform(data_test[['statuses_count', 'followers_count', 'friends_count','favourites_count','listed_count']])

In [114]:
data_test_1=data_test[['geo_enabled','is_translator','profile_banner_url_dup','following','lang','statuses_count', 'followers_count', 'friends_count','favourites_count','listed_count']]
one_hot = pd.get_dummies(data_test_1['lang'])
data_test_1= pd.concat([data_test_1, one_hot], axis=1)
data_test_1 = data_test_1.drop('lang', axis=1)
data_test_1.drop(['en-GB'],axis=1,inplace=True)
data_test_1 = data_test_1[data_test_1.columns[~data_test_1.columns.isin(["label"])]]
X_train_00.drop(['ca','nl','pl','ru'],axis=1,inplace=True)
logreg = LogisticRegression(C=10, penalty='l2', max_iter=10000)
logreg.fit(X_train_00, y_train_00)
y_pred = logreg.predict(data_test_1)


In [119]:
y_pred

array(['human', 'human', 'human', 'human', 'human', 'human', 'bot',
       'human', 'human', 'human', 'human', 'bot', 'human', 'human',
       'human', 'human', 'human', 'human', 'human', 'bot', 'human',
       'human', 'human', 'bot', 'human', 'human', 'bot', 'bot', 'human',
       'bot', 'human', 'human', 'human', 'human', 'human', 'human',
       'human', 'bot', 'human', 'human', 'human', 'bot', 'human', 'human',
       'human', 'bot', 'human', 'human', 'bot', 'human', 'human', 'human',
       'bot', 'human', 'human', 'human', 'bot', 'human', 'human', 'human',
       'human', 'human', 'bot', 'human', 'human', 'human', 'human', 'bot',
       'human', 'human', 'human', 'human', 'human', 'human', 'bot', 'bot',
       'human', 'human', 'human', 'bot', 'human', 'bot', 'bot', 'human',
       'human', 'human', 'human', 'bot', 'human', 'bot', 'human', 'human',
       'human', 'bot', 'human', 'human', 'human', 'human', 'human',
       'human', 'bot', 'bot', 'human', 'human', 'human', 'human'

In [115]:
#保存下来
data_final=data_test
data_final['label'] = y_pred
import json
def save_data(df, path):
    user_list = df.to_dict(orient='records')
    with open(path, 'w') as json_file:
        json.dump({"user": user_list}, json_file)

save_data(data_final, './data/test.json')

C:\Users\CindyWu\AppData\Local\Temp\ipykernel_2204\4104750091.py:6: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  user_list = df.to_dict(orient='records')


In [118]:
import pandas as pd

def load_data(path):
    df = pd.read_json(path)
    df_expanded = df['user'].apply(lambda x: pd.Series(x))
    df = pd.concat([df.drop('user', axis=1), df_expanded], axis=1)
    return df

df = load_data("./data/test.json")
df_raw = load_data("./data/rawtest.json")

assert set(df["label"]) == {"human", "bot"}
assert list(df["id"]) == list(df_raw["id"])

print("检测通过")

检测通过
